In [1]:
#Imports
import pandas as pd
import pyodbc

In [2]:
#Def of function to select data from database
def select_to_DF(server, DB, query):
    sqlCommand="Driver={SQL Server};Server=%s;Database=%s;Trusted_Connection=yes;" %(server,DB)
    connection = pyodbc.connect(sqlCommand)
    sqlCommand = query
    df = pd.read_sql(query, connection)
    connection.commit()
    connection.close()
    return df

server = 'DESKTOP-T7QEG5U\SQLEXPRESS'
DB = 'PortfolioProject'

In [3]:
query=f"""
Select SUM(new_cases) as TotalCases, SUM(cast (new_deaths as int)) as TotalDeaths,
SUM(cast(new_deaths as int))/SUM(new_cases)*100 as InfectedDeathPercentage
From PortfolioProject..CovidDeaths
Where continent is not null
Order by 1,2
      """
Total_Global_Numbers=select_to_DF(server,DB,query)
Total_Global_Numbers.to_excel('Table_1.xlsx', index=False)

In [4]:
query=f"""
Select location, SUM(cast(new_deaths as int)) as TotalDeathCount
From PortfolioProject..CovidDeaths
Where continent is null and location in (select continent from PortfolioProject..CovidDeaths)
Group by location
Order by TotalDeathCount desc
      """
Continents_Total_Deaths=select_to_DF(server,DB,query)
Continents_Total_Deaths.to_excel('Table_2.xlsx', index=False)

In [6]:
query=f"""
Select location,population,MAX(total_cases) as HighestInfectionCount, MAX((total_cases/population))*100 as PercentInfected 
From PortfolioProject..CovidDeaths
Where continent is not null and population is not null
Group by location,population
Having MAX(total_cases) is not null
Order by PercentInfected desc

      """
Counrties_Max_Infection=select_to_DF(server,DB,query)
Counrties_Max_Infection.to_excel('Table_3.xlsx', index=False)

In [9]:
query=f"""
Select location, population, date, ISNULL(HighestInfectionCount, 0 ) as HighestInfectionCount , ISNULL(PercentInfected, 0 ) as  PercentInfected
From (
Select location, population, date, MAX(total_cases) as HighestInfectionCount, MAX(total_cases/population)*100 as PercentInfected 
From PortfolioProject..CovidDeaths
Where continent is not null
Group by location, population, date
) a
Order by PercentInfected desc 
      """
Counrties_Percent_Infected_by_Date=select_to_DF(server,DB,query)
Counrties_Percent_Infected_by_Date.to_excel('Table_4.xlsx', index=False)